In [10]:
using LinearAlgebra
using Flux

In [11]:
using Symbolics
using SymbolicNumericIntegration

In [14]:
@variables t

1-element Vector{Num}:
 t

In [23]:
function test(tt)
    sx = sin(tt)
    gx = integrate(sx)
    gxx = substitute.(gx, tt=>[0.1, 0.2, 0.3])
    return gxx
end

test (generic function with 2 methods)

In [24]:
gtt = test(t)

MethodError: MethodError: no method matching cos(::Vector{Float64})

Closest candidates are:
  cos(!Matched::T) where T<:Union{Float32, Float64}
   @ Base special/trig.jl:98
  cos(!Matched::Lazy.List, !Matched::Lazy.List...)
   @ Lazy ~/.julia/packages/Lazy/9Xnd3/src/Lazy.jl:20
  cos(!Matched::SymbolicUtils.BasicSymbolic{<:Number})
   @ SymbolicUtils ~/.julia/packages/SymbolicUtils/H684H/src/methods.jl:83
  ...


In [21]:
gxx = @code_warntype test(t)

MethodInstance for test(::Num)
  from test(tt) @ Main ~/Documents/Codes/julia/RNN-RF/test_M1MBA.ipynb:1
Arguments
  #self#::Core.Const(test)
  tt::Num
Locals
  gxx::Any
  gx::Tuple{Any, Any, Any}
  sx::Num
Body::Any
1 ─      (sx = Main.sin(tt))
│        (gx = Main.integrate(sx))
│   %3 = Main.substitute::Core.Const(SymbolicUtils.substitute)
│   %4 = gx::Tuple{Any, Any, Any}
│   %5 = (2 * Main.pi)::Core.Const(6.283185307179586)
│   %6 = (0:0.1:%5)::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}
│   %7 = (tt => %6)::Pair{Num, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}
│   %8 = Base.broadcasted(%3, %4, %7)::Base.Broadcast.Broadcasted{Style, Nothing, typeof(substitute)} where Style<:Union{Nothing, Base.Broadcast.BroadcastStyle}
│        (gxx = Base.materialize(%8))
└──      return gxx



In [27]:
model = Dense(zeros(Float32, 7, 60))

Dense(60 => 7)      # 427 parameters  (all zero)

In [28]:
typeof(model)

Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}

In [29]:
model2 = Chain(Dense(60, 64), Tuple(Chain(Parallel(+, Chain(BatchNorm(64), Dense(64, 64, relu)),Dense(64, 64, relu)), identity) for i in 1:2)..., Flux.flatten, Flux.Parallel(vcat, Dense(64, 6, tanh), Dense(64, 1, tanh)))

Chain(
  Dense(60 => 64),                      # 3_904 parameters
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(64),                  # 128 parameters, plus 128
        Dense(64 => 64, relu),          # 4_160 parameters
      ),
      Dense(64 => 64, relu),            # 4_160 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(64),                  # 128 parameters, plus 128
        Dense(64 => 64, relu),          # 4_160 parameters
      ),
      Dense(64 => 64, relu),            # 4_160 parameters
    ),
    identity,
  ),
  Flux.flatten,
  Parallel(
    vcat,
    Dense(64 => 6, tanh),               # 390 parameters
    Dense(64 => 1, tanh),               # 65 parameters
  ),
)         # Total: 18 trainable arrays, 21_255 parameters,
          # plus 4 non-trainable, 256 parameters, summarysize 85.824 KiB.

In [30]:
typeof(model2)

Chain{Tuple{Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}, Chain{Tuple{Parallel{typeof(+), Tuple{Chain{Tuple{BatchNorm{typeof(identity), Vector{Float32}, Float32, Vector{Float32}}, Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}}}, Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}}}, typeof(identity)}}, Chain{Tuple{Parallel{typeof(+), Tuple{Chain{Tuple{BatchNorm{typeof(identity), Vector{Float32}, Float32, Vector{Float32}}, Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}}}, Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}}}, typeof(identity)}}, typeof(Flux.flatten), Parallel{typeof(vcat), Tuple{Dense{typeof(tanh), Matrix{Float32}, Vector{Float32}}, Dense{typeof(tanh), Matrix{Float32}, Vector{Float32}}}}}}

In [31]:
mutable struct test_model
    model::Chain
end

In [32]:
mm = test_model(model2)

test_model(Chain(Dense(60 => 64), Chain(Parallel(+, Chain(BatchNorm(64), Dense(64 => 64, relu)), Dense(64 => 64, relu)), identity), Chain(Parallel(+, Chain(BatchNorm(64), Dense(64 => 64, relu)), Dense(64 => 64, relu)), identity), flatten, Parallel(vcat, Dense(64 => 6, tanh), Dense(64 => 1, tanh))))

In [34]:
mutable struct test_model2
    model::Chain
    storage::Dict{Int, Chain}
end

In [35]:
mm2 = test_model2(model2, Dict())

test_model2(Chain(Dense(60 => 64), Chain(Parallel(+, Chain(BatchNorm(64), Dense(64 => 64, relu)), Dense(64 => 64, relu)), identity), Chain(Parallel(+, Chain(BatchNorm(64), Dense(64 => 64, relu)), Dense(64 => 64, relu)), identity), flatten, Parallel(vcat, Dense(64 => 6, tanh), Dense(64 => 1, tanh))), Dict{Int64, Chain}())

In [39]:
typeof(mm2.storage)

Dict{Int64, Chain}

In [1]:
test = Dict()

Dict{Any, Any}()

In [2]:
isempty(test)

true

In [5]:
length(test)

1

In [4]:
test[3] = 4

4

In [2]:
visit = [2, 3, 5, 6, 1]

5-element Vector{Int64}:
 2
 3
 5
 6
 1

In [21]:
using StatsBase
using Distributions

In [3]:
p = softmax(visit)

5-element Vector{Float64}:
 0.012695212984686215
 0.034509166764689854
 0.25499016915164646
 0.6931351432406189
 0.004670307858358804

In [13]:
ps = [0.2, 0.3, 0.5]

3-element Vector{Float64}:
 0.2
 0.3
 0.5

In [15]:
rand(ps)

0.5

In [27]:
rand(Categorical(p))

4

In [25]:
input = 10
middle = 20
output = 8

8

In [21]:
Tuple(i for i in 1:5)

(1, 2, 3, 4, 5)

In [20]:
model = Chain(Dense(input, middle), [Chain(Parallel(+, Chain(BatchNorm(middle), Dense(middle, middle, relu)),Dense(middle, middle, relu)), identity) for i in 1:3], Flux.flatten ,Dense(middle, output, tanh))

Chain(
  Dense(10 => 20),                      # 220 parameters
  Array(
    Chain(
      Parallel(
        +,
        Chain(
          BatchNorm(20),                # 40 parameters, plus 40
          Dense(20 => 20, relu),        # 420 parameters
        ),
        Dense(20 => 20, relu),          # 420 parameters
      ),
      identity,
    ),
    Chain(
      Parallel(
        +,
        Chain(
          BatchNorm(20),                # 40 parameters, plus 40
          Dense(20 => 20, relu),        # 420 parameters
        ),
        Dense(20 => 20, relu),          # 420 parameters
      ),
      identity,
    ),
    Chain(
      Parallel(
        +,
        Chain(
          BatchNorm(20),                # 40 parameters, plus 40
          Dense(20 => 20, relu),        # 420 parameters
        ),
        Dense(20 => 20, relu),          # 420 parameters
      ),
      identity,
    ),
  ),
  Flux.flatten,
  Dense(20 => 8, tanh),                 # 168 parameters
)         # Total: 22 tr